
<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>



In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/CS 1: Quora Question Pair Similarity/questions.csv', nrows=100000)

df['question1'] = df['question1'].apply( lambda x: str(x))
df['question2'] = df['question2'].apply( lambda x: str(x))


In [0]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions = list(df['question1']) + list(df['question2'])
tfidf=TfidfVectorizer(lowercase=False)
tfidf.fit_transform(questions)

word2tfidf = dict( zip(tfidf.get_feature_names(),tfidf.idf_))
print(word2tfidf)

{'00': 10.115035192159358, '000': 7.325544659116973, '0001': 12.512930464957728, '000K': 12.512930464957728, '000WebHost': 12.512930464957728, '000ft': 12.512930464957728, '000s': 12.512930464957728, '000webhost': 12.512930464957728, '001': 12.512930464957728, '0015': 12.512930464957728, '0019': 12.512930464957728, '002': 12.512930464957728, '0021': 12.512930464957728, '0035': 12.512930464957728, '003SC': 12.512930464957728, '0051': 12.512930464957728, '008': 12.512930464957728, '008801703772104': 12.512930464957728, '00am': 12.512930464957728, '00g': 12.512930464957728, '01': 11.008853068181454, '012': 12.512930464957728, '013J': 12.512930464957728, '02': 11.819783284397783, '025': 12.512930464957728, '03': 11.596639733083574, '032': 12.512930464957728, '03g': 12.512930464957728, '04': 9.987201820649473, '04009': 12.512930464957728, '05': 12.107465356849564, '050': 12.512930464957728, '05246': 12.512930464957728, '054': 12.512930464957728, '06': 11.596639733083574, '066': 12.107465356

# **weighted word2vec code understanding**

In [0]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
#f=df['question1'][0]
#f

In [0]:
#nlp = spacy.load('en_core_web_lg')
#doc1 = nlp(f)
#print(doc1[0].vector)
#print(len(doc1))
#print(len(doc1[0].vector))

In [0]:
#mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
#print(mean_vec1.shape)
#mean_vec1

In [0]:
#doc1[0].vector.shape

In [0]:
"""for word1 in doc1:
  # word2vec
  vec1 = word1.vector
  #print(vec1.shape)
        # fetch df score
  try:
    idf = word2tfidf[str(word1)]
  except:
    idf = 0
        # compute final vec
  mean_vec1 += vec1 * idf
mean_vec1 = mean_vec1.mean(axis=0)
mean_vec1.shape """

'for word1 in doc1:\n  # word2vec\n  vec1 = word1.vector\n  #print(vec1.shape)\n        # fetch df score\n  try:\n    idf = word2tfidf[str(word1)]\n  except:\n    idf = 0\n        # compute final vec\n  mean_vec1 += vec1 * idf\nmean_vec1 = mean_vec1.mean(axis=0)\nmean_vec1.shape '

In [0]:
#df.question1.values.tolist()

**End of Understandng**

# TF-IDF Weighted Word2Vec

In [0]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

vecs1 = []

for qu1 in tqdm(list(df['question1'])):
  doc1 = nlp(qu1)
  mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
  
  for word in doc1:
    vec1 = word.vector
    try:
      tf = word2tfidf[str(word)]
    except:
      tf=0
    mean_vec1 += vec1*tf
  mean_vec1 = mean_vec1.mean(axis=0)
  vecs1.append(mean_vec1)

df['q1_feats_m'] = list(vecs1)


100%|██████████| 100000/100000 [17:36<00:00, 94.64it/s]


In [0]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
  doc2 = nlp(qu2)
  mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
  for word in doc2:
    vec2 = word.vector
    try:
      tf = word2tfidf[str(word)]
    except:
      tf=0
    mean_vec2 += vec2*tf
  mean_vec2 =mean_vec2.mean(axis=0)
  vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 100000/100000 [17:51<00:00, 88.18it/s]


In [0]:
dfwpre = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CS 1: Quora Question Pair Similarity/df_fe_without_preprocessing_train.csv', encoding='latin-1')

dfnlp = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CS 1: Quora Question Pair Similarity/nlp_features_train.csv', sep='\t', encoding= 'latin-1')

In [0]:
df['id'].head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
dfwpre.info()
dfnlp.info()

# Dropping Columns and Merging dataframes

In [0]:
df1 = dfnlp.drop(['qid1', 'qid2', 'question1', 'question2'], axis=1)
df2 = dfwpre.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [0]:

df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.666656,0.666656,0.999980,0.833319,0.749994,0.642853,0.0,0.0,2.0,13.0,100,93,93,96,0.948276
1,1,0,0.199996,0.099999,0.499975,0.199996,0.199998,0.153845,0.0,0.0,3.0,11.5,86,63,66,61,0.173077
2,2,0,0.166664,0.166664,0.249994,0.124998,0.199998,0.142856,0.0,0.0,4.0,12.0,63,63,43,38,0.150000
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,10.0,28,24,9,8,0.039216
4,4,0,0.166664,0.099999,0.999900,0.333322,0.285710,0.153845,0.0,0.0,6.0,10.0,67,47,35,51,0.150000


In [0]:
df2.head()

In [0]:
df3.head()

,id,q1_feats_m,q2_feats_m
0,0,"[-5.694012746214867, 17.548630751669407, 4.776...","[0.45537663996219635, 14.135056056082249, -0.5..."
1,1,"[8.92591068148613, 12.963734991848469, 18.4354...","[4.289293080568314, 9.69414658844471, 19.77851..."
2,2,"[1.0144839137792587, 16.109177358448505, -8.16...","[-17.377070650458336, 17.485377540811896, -9.2..."
3,3,"[-4.962438712827861, 17.069576263427734, -15.4...","[3.918512497097254, 2.4612065330147743, -1.866..."
4,4,"[-11.315619349479675, 19.77099174261093, -4.20...","[-5.365081623196602, 1.7945491075515747, 1.824..."


In [0]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 28
Number of features in preprocessed dataframe : 601
Number of features in question1 w2v  dataframe : 301
Number of features in question2 w2v  dataframe : 301
Number of features in final dataframe  : 1231


In [0]:
df3_q1['id'] = df1['id']
df3_q2['id'] = df1['id']
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')
result  = df1.merge(df2, on='id',how='left')
result.to_csv('final_features.csv', sep='\t')
from google.colab import files
#files.download("final_features.csv")

In [0]:
result.head(2)

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_total,word_share,freq_q1+q2,freq_q1-q2,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,...,260_y,261_y,262_y,263_y,264_y,265_y,266_y,267_y,268_y,269_y,270_y,271_y,272_y,273_y,274_y,275_y,276_y,277_y,278_y,279_y,280_y,281_y,282_y,283_y,284_y,285_y,286_y,287_y,288_y,289_y,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,0,0.666656,0.666656,0.999980,0.833319,0.749994,0.642853,0.0,0.0,2.0,13.0,100,93,93,96,0.948276,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,-5.694013,17.548631,4.776329,8.033126,20.358069,-5.519559,-4.142384,-2.759443,8.058761,146.981455,-45.384338,-3.160134,...,-1.467404,6.082916,8.532413,2.703683,6.031922,-28.902220,6.008976,0.712155,26.287629,3.604746,-1.977775,2.172850,-4.942044,-16.320698,-7.341654,-4.679539,-2.096015,3.857902,13.305752,1.062959,0.573487,9.400508,-3.007137,4.013453,1.190026,-13.578231,2.687878,-4.606748,3.270668,3.522743,-17.761312,7.219609,1.539488,-7.567536,0.419017,-11.864802,-11.312722,2.024517,3.534485,12.025257
1,1,0,0.199996,0.099999,0.499975,0.199996,0.199998,0.153845,0.0,0.0,3.0,11.5,86,63,66,61,0.173077,1,1,51,88,8,13,4.0,20.0,0.200000,2,0,8.925911,12.963735,18.435409,-2.228467,-15.298821,-1.978872,8.841655,-19.795839,-20.364966,15.598965,-5.048854,-11.850565,...,3.407139,-3.541979,7.184326,2.175545,-6.771205,-17.806993,6.423980,8.640396,-25.227398,8.180302,-31.136884,6.396593,2.739909,22.578386,3.867989,15.483774,-2.239942,4.921820,-15.699003,6.083096,-7.596422,-9.891302,1.051400,4.829672,-13.041017,-4.038531,-0.335275,-1.533693,-2.889977,1.070658,22.028073,3.585006,-4.792569,7.020478,33.634991,6.137379,-27.196006,12.500474,-4.343158,33.140916


# Download the file to google drive

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
uploaded = drive.CreateFile({'title': 'final_features.csv'})
uploaded.SetContentFile('final_features.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1MNxW3RNKuK7S3HAFHD3fM1k1PKAoyOLU


In [0]:
d=pd.read_csv('/content/drive/My Drive/final_features.csv', sep = '\t')
d.head(2)

,Unnamed: 0,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_total,word_share,freq_q1+q2,freq_q1-q2,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,...,260_y,261_y,262_y,263_y,264_y,265_y,266_y,267_y,268_y,269_y,270_y,271_y,272_y,273_y,274_y,275_y,276_y,277_y,278_y,279_y,280_y,281_y,282_y,283_y,284_y,285_y,286_y,287_y,288_y,289_y,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,0,0,0.666656,0.666656,0.999980,0.833319,0.749994,0.642853,0.0,0.0,2.0,13.0,100,93,93,96,0.948276,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,-5.694013,17.548631,4.776329,8.033126,20.358069,-5.519559,-4.142384,-2.759443,8.058761,146.981455,-45.384338,...,-1.467404,6.082916,8.532413,2.703683,6.031922,-28.902220,6.008976,0.712155,26.287629,3.604746,-1.977775,2.172850,-4.942044,-16.320698,-7.341654,-4.679539,-2.096015,3.857902,13.305752,1.062959,0.573487,9.400508,-3.007137,4.013453,1.190026,-13.578231,2.687878,-4.606748,3.270668,3.522743,-17.761312,7.219609,1.539488,-7.567536,0.419017,-11.864802,-11.312722,2.024517,3.534485,12.025257
1,1,1,0,0.199996,0.099999,0.499975,0.199996,0.199998,0.153845,0.0,0.0,3.0,11.5,86,63,66,61,0.173077,1,1,51,88,8,13,4.0,20.0,0.200000,2,0,8.925911,12.963735,18.435409,-2.228467,-15.298821,-1.978872,8.841655,-19.795839,-20.364966,15.598965,-5.048854,...,3.407139,-3.541979,7.184326,2.175545,-6.771205,-17.806993,6.423980,8.640396,-25.227398,8.180302,-31.136884,6.396593,2.739909,22.578386,3.867989,15.483774,-2.239942,4.921820,-15.699003,6.083096,-7.596422,-9.891302,1.051400,4.829672,-13.041017,-4.038531,-0.335275,-1.533693,-2.889977,1.070658,22.028073,3.585006,-4.792569,7.020478,33.634991,6.137379,-27.196006,12.500474,-4.343158,33.140916
